In [1]:
import cfbd
import json
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import json

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'j5Z7/ASYjq4l6PMJFJpY7WxwO5BtbY/U2TFBOVdziGKRvjI8WajGUnVRiUBGrIX8'
configuration.api_key_prefix['Authorization'] = 'Bearer'

games_api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
metrics_api_instance = cfbd.MetricsApi(cfbd.ApiClient(configuration))
plays_api_instance = cfbd.PlaysApi(cfbd.ApiClient(configuration))

In [21]:
YEAR = 2023

# TODO: not just michigan
# TODO: timezones
# TODO: output to file

# for team in ('michigan', 'georgia', 'ohio state', 'alabama', 'colorado', 'florida state', 'clemson', 'utah', 'usc', 'washington', 'oregon'):
# NOTE: there are some games without any win probability data for some reason
for team in ('minnesota', 'kansas', 'utah', 'iowa', 'michigan state', 'louisiana state', 'south carolina', 'oklahoma state', 'wyoming', 'washington state'):
    for season_type in ('regular', 'postseason'):
        games = games_api_instance.get_games(year=YEAR, team=team, season_type=season_type)
        for game in games:
            output = []
            print(f'id: {game._id} home: {game._home_team} away: {game._away_team}')
            wp = metrics_api_instance.get_win_probability_data(game_id=game._id)
            if not wp:
                continue
            print(f'initial win prob: {wp[0]._home_win_prob:.3f}, final: {wp[-1]._home_win_prob:.3f}')
            plays = plays_api_instance.get_plays(year=YEAR, week=game._week, team=game._home_team, season_type=season_type)
            for i in range(len(wp)):
                j = i
                while (j < len(plays) and wp[i]._play_id != plays[j]._id):
                    j += 1

                dt_utc = None
                if j == len(plays) or plays[j]._wallclock is not None:
                    if j == len(plays) or ():
                        dt_utc = output[-1]['utc'] + 1
                    else:
                        dt_utc = datetime.fromisoformat(plays[j]._wallclock).astimezone(tz=None).timestamp()
                    # Don't allow timestamp for a given play to be before a previous play!
                    if i > 1 and dt_utc < output[-1]['utc']:
                        dt_utc = output[-1]['utc'] + 1
                    if (j == len(plays) or wp[i]._play_id == plays[j]._id):
                        agg_play = {
                            'play_number' : i,
                            'play_id' : wp[i]._play_id,
                            'home_win_prob' : wp[i]._home_win_prob,
                            # 'wallclock' : plays[j]._wallclock,
                            'utc': int(dt_utc),
                        }
                        output.append(agg_play)
            with open(f'win_probs/{game._home_team} vs {game._away_team}.json', 'w') as f:
                json.dump(output, f)
            # Plotting win percentage
    #         time = [play['utc'] for play in output]
    #         wp_list = [play['home_win_prob'] for play in output]
    #         plot = plt.plot(time, wp_list)
    #         plt.xlabel('UTC time')
    #         plt.ylabel('Home Win Prob')
    #         plt.title(f'(H) {game._home_team} vs (A) {game._away_team}')
    #         plt.show()

    #         plot = plt.plot(range(len(output)), wp_list)
    #         plt.xlabel('Play Number')
    #         plt.ylabel('Home Win Prob')
    #         plt.title(f'(H) {game._home_team} vs (A) {game._away_team}')
    #         plt.show()
#         break

id: 401520148 home: Minnesota away: Nebraska
initial win prob: 0.720, final: 1.000
id: 401520204 home: Minnesota away: Eastern Michigan
initial win prob: 0.448, final: 1.000
id: 401520239 home: North Carolina away: Minnesota
initial win prob: 0.533, final: 1.000
id: 401520264 home: Northwestern away: Minnesota
initial win prob: 0.382, final: 1.000
id: 401520284 home: Minnesota away: Louisiana
initial win prob: 0.448, final: 1.000
id: 401520303 home: Minnesota away: Michigan
initial win prob: 0.534, final: 0.000
id: 401520338 home: Iowa away: Minnesota
initial win prob: 0.534, final: 0.000
id: 401520354 home: Minnesota away: Michigan State
initial win prob: 0.534, final: 1.000
id: 401520370 home: Minnesota away: Illinois
initial win prob: 0.369, final: 0.000
id: 401520395 home: Purdue away: Minnesota
initial win prob: 0.431, final: 1.000
id: 401520415 home: Ohio State away: Minnesota
initial win prob: 0.448, final: 1.000
id: 401520436 home: Minnesota away: Wisconsin
initial win prob: 0.